# Clave pública

In [99]:
raw = '041ba73b45d7d1948351b92073aef3fb77af348815ae9edbe6a29d98d5d7d3de1165dd7b1fb40ee534c0fba27def07cdfa64ae45522ddd4c4338a169f4606cac09'
len(raw)

130

In [100]:
cabecera = raw[0:2]
cabecera

'04'

In [101]:
raw2 = raw[2:]
mitad = len(raw2)//2

In [102]:
Qx = int(raw2[:mitad], 16)
Qy = int(raw2[mitad:], 16)

# Datos de la curva

Signature Algorithm: ecdsa_secp256r1_sha256

## Datos de la curva 256 del documento

In [103]:
p = 115792089210356248762697446949407573530086143415290314195533631308867097853951
n = 115792089210356248762697446949407573529996955224135760342422259061068512044369
a = -3
b = 0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b
Gx = 0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296
Gy = 0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5

## Firma

In [104]:
raw = '3046022100914009da3d6ed684b32c63afe9c02eef32baa9ea6555ac50fad395bade9b5ae8022100cfeffb57304c724b15ab4fea764df1223f661846cb69df1e69f2fb7c9d992699'

### Extraer cabecera

In [105]:
print(raw[0:2])
raw = raw[2:]
print(raw)

30
46022100914009da3d6ed684b32c63afe9c02eef32baa9ea6555ac50fad395bade9b5ae8022100cfeffb57304c724b15ab4fea764df1223f661846cb69df1e69f2fb7c9d992699


### Comprobar longiutd de la cadena

In [106]:
longitud = raw[:2]
raw = raw[2:]
len(raw)//2 == int(longitud, 16)

True

### Obtener f1

In [107]:
print(raw[:2])
raw = raw[2:]

02


In [108]:
longitud = int(raw[:2], 16)
raw = raw[2:]
print(longitud)

33


In [109]:
f1 = raw[:longitud*2]
len(f1)

66

In [110]:
f1

'00914009da3d6ed684b32c63afe9c02eef32baa9ea6555ac50fad395bade9b5ae8'

In [111]:
raw = raw[longitud*2:]

### Obtener f2

In [112]:
print(raw[:2])
raw = raw[2:]

02


In [113]:
longitud = int(raw[:2], 16)
raw = raw[2:]
print(longitud)

33


In [114]:
f2 = raw[:longitud*2]
len(f2)

66

In [115]:
f2

'00cfeffb57304c724b15ab4fea764df1223f661846cb69df1e69f2fb7c9d992699'

In [116]:
raw = raw[longitud*2:]

### Raw deberia estar vacia

In [117]:
raw

''

# Mensaje firmado

## Preambulo

In [118]:
primero = '20' * 64
primero

'20202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020'

In [119]:
frase = 'TLS 1.3, server CertificateVerify'
segundo = ''
for letra in frase:
    numero = hex(ord(letra))[2:]
    numero = '0'*(2 - len(numero)) + numero
    segundo += numero
segundo

'544c5320312e332c20736572766572204365727469666963617465566572696679'

In [120]:
tercero = '00'

In [121]:
preambulo = primero + segundo + tercero
preambulo

'20202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020544c5320312e332c2073657276657220436572746966696361746556657269667900'

## Mensaje firmado

In [122]:
with open('./Wikipedia/mensaje.bin', 'rb') as f:
    mensaje = f.read()

In [123]:
from Crypto.Hash import SHA384
h = SHA384.new()
h.update(mensaje)
mensaje384 = h.hexdigest()

In [124]:
mensaje384

'34d37fd950b10f7fcc91eccf527459627d8c69b1ce6915ab5f7c460e1e7ce00359eb46c27274fe83f7fb07b2e8f583dc'

In [125]:
from Crypto.Hash import SHA256
h = SHA256.new()
h.update((preambulo + mensaje384).encode())
m = h.hexdigest()

In [126]:
m

'570342f7cdc72d31cc34d2b459e35070a817cb5c472c9d0e4ce208def302d178'

# Verificacion firma

## Calculo pesos

In [129]:
w1 = (int(m, 16) * pow(int(f2, 16), -1, n)) % n
w1

68418182177657162295548549195812790816093669873385508137672486405183595270022

In [130]:
w2 = (int(f1, 16) * pow(int(f2, 16), -1, n)) % n
w2

89067318584195525323085404731611433077958851466746282182737511975670725719832

## Creacion curva y puntos

In [131]:
from Crypto.PublicKey.ECC import *
curva = 'p256'

In [132]:
G = EccPoint(Gx, Gy, curve=curva)
Q = EccPoint(Qx, Qy, curve=curva)

## Calculos en la curva

In [133]:
E = w1 * G + w2 * Q

In [134]:
x0, y0 = E.x, E.y

In [136]:
comprobacion = int(f1, 16) % n

In [137]:
comprobacion == x0

False

In [138]:
E.x

Integer(7788894992874896690883675213399901128045469013709283178582614051176394101900)

In [140]:
comprobacion

65698509256174583349959437356701870316724411153934220427135920273881720117992